## Spectrograms of Real Data - tidied up
   Here results are presented in cells
   Signals from all channels are analysed here
    

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
%matplotlib inline

#read in n lines of data         X0   Y0   X1   Y1
#the data is the signal in Volts, but the callibration is not clear yet so take as arbitrary units
#skips the first 7 rows and places the next 10000 in a numpy array
data43 = np.loadtxt(fname = 'dataFiles/run-43', skiprows = 7, max_rows = 4000000)

#time step (seconds)
dt = 0.01

#Make complex signal 1
#X0 = data43[:,0]
#Y0 = data43[:,1]
#iY0 = 1j*Y0

#Make complex signal 1
#X1 = data43[:,2]
#Y1 = data43[:,3]
#iY1 = 1j*Y1
#combine x+iy to make complex signal
#sig_0 = X0+iY0
sig_1 = X1+iY1

NameError: name 'X1' is not defined

### Spectrogram

In [5]:
#SIGNAL 0
plt.figure(figsize=(20,10))

spectrum0, freqs0, t_bins0, im0 = signal.spectrogram(x=sig_0, Fs=1/dt, NFFT = 2000, sides = 'twosided')

plt.ylabel("Frequency Hz",fontsize=23)
plt.xlabel("Time s",fontsize=23)
cbar = plt.colorbar(im0);
plt.xticks(fontsize=23)
plt.yticks(fontsize=23)
cbar.ax.tick_params(labelsize=20)
plt.title('Unnormalised spectrogram 0',fontsize=25)



TypeError: spectrogram() got an unexpected keyword argument 'Fs'

<Figure size 1440x720 with 0 Axes>

In [ ]:
#SIGNAL 1
plt.figure(figsize=(20,10))

spectrum1, freqs1, t_bins1, im1 = plt.specgram(x=sig_1, Fs=1/dt, NFFT = 2000, sides = 'twosided')

plt.ylabel("Frequency Hz",fontsize=23)
plt.xlabel("Time s",fontsize=23)
cbar = plt.colorbar(im1);
plt.xticks(fontsize=23)
plt.yticks(fontsize=23)
cbar.ax.tick_params(labelsize=20)
plt.title('Unnormalised spectrogram 1',fontsize=25)



In [ ]:
#SIGNAL 0
#normalise spectrum powers
#Median normalisation across individual frequencies
norm_spectrum0 = np.ones(np.shape(spectrum0))
for row in range(len(spectrum0)):
    median_row =  np.median(spectrum0[row])
    norm_spectrum0[row] = spectrum0[row]/median_row

    #plotting spectrogram with normalised powers
plt.figure(figsize=(20,10))
plt.pcolormesh(t_bins0, freqs0, 10 * np.log10(norm_spectrum0))

plt.ylabel("Frequency Hz",fontsize=23)
plt.xlabel("Time s",fontsize=23)
cbar = plt.colorbar();
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.title('Normalised spectrogram 0',fontsize=25)


cbar.ax.tick_params(labelsize=20)

In [ ]:
#SIGNAL 1

#normalise spectrum powers
#Median normalisation across individual frequencies
norm_spectrum1 = np.ones(np.shape(spectrum1))
for row in range(len(spectrum1)):
    median_row =  np.median(spectrum1[row])
    norm_spectrum1[row] = spectrum1[row]/median_row

    #plotting spectrogram with normalised powers
plt.figure(figsize=(20,10))
plt.pcolormesh(t_bins1, freqs1, 10 * np.log10(norm_spectrum1))

plt.ylabel("Frequency Hz",fontsize=23)
plt.xlabel("Time s",fontsize=23)
cbar = plt.colorbar();
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.title('Normalised spectrogram 1',fontsize=25)


cbar.ax.tick_params(labelsize=20)

### Frequency Slicing

In [ ]:
#Frequency slicing
#need to take frequencies in the resonant mode roughly 2-4Hz and one ouside this range eg. 30-40Hz

def freq_slice(min_freq, max_freq, freqs, t_bins, norm_spectrum,title):
    '''function slices the array containing the frequency spectrum using the min and max boundaries,
    and the corresponding spectrum powers. It plots the slices specgram and returns slice spectrum.'''
    
    #find the elements in the array in the HZ range
    slice_freqs = freqs[(min_freq<freqs) & (freqs<max_freq)]
    #and their indicies
    slice_indices = np.where((min_freq<freqs) & (freqs<max_freq))

    #need the spectrum powers corresponding to these freqs
    slice_spectrum = norm_spectrum[slice_indices]
    #res_mode_tbins = t_bins[res_mode_indices]
    
    #This doesn't really need plotted
    
    #now plot the spectrogram for this range
    plt.pcolormesh(t_bins, slice_freqs, 10 * np.log10(slice_spectrum))
    
    
    plt.ylabel("Frequency Hz",fontsize=23)
    plt.xlabel("Time s",fontsize=23)
    cbar = plt.colorbar();
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    cbar.ax.tick_params(labelsize=20)
    plt.title(title)
    
    return slice_spectrum

plt.figure(figsize=(20,10))
#Call function for resonant mode slice
plt.subplot(211)
res_slice_spec0 = freq_slice(-3,-2, freqs=freqs0, t_bins=t_bins0, norm_spectrum=norm_spectrum0,title = 'Normalised spectrum 0, 2-3Hz')

#Call function for background
plt.subplot(212)
bkgd_slice_spec0 = freq_slice(20,30, freqs=freqs0, t_bins=t_bins0, norm_spectrum=norm_spectrum0,title='Normalised spectrum 0, 20-30Hz')


In [ ]:
#SIGNAL 1

plt.figure(figsize=(20,10))
#Call function for resonant mode slice
plt.subplot(211)
res_slice_spec1 = freq_slice(2,3, freqs=freqs1, t_bins=t_bins1, norm_spectrum=norm_spectrum1,title = 'Normalised spectrum 1, 2-3Hz')

#Call function for background
plt.subplot(212)
bkgd_slice_spec1 = freq_slice(20,30, freqs=freqs1, t_bins=t_bins1, norm_spectrum=norm_spectrum1,title='Normalised spectrum 1, 20-30Hz')


### Differencing

In [ ]:
#need array containing the differences between successive columns of spectrogram data in spectrum

def difference(power_spectrum, spec_ydim, spec_xdim):
    
    #difference array will have one fewer column than the spectrogram
    diff_spec = np.ones((spec_ydim, spec_xdim -1))

    for i in range(spec_ydim):
        for j in range(spec_xdim-1):
            diff_spec[i,j] = power_spectrum[i,j+1] - power_spectrum[i,j]
    
    #(square differences) <- changed to absolute value
    diff_spec = np.absolute(diff_spec)
    
    return diff_spec

#resonant mode slices:
#SIGNAL0
res_diff_spec0 = difference(res_slice_spec0, np.shape(res_slice_spec0)[0], np.shape(res_slice_spec0)[1])
bkgd_diff_spec0 = difference(bkgd_slice_spec0,np.shape(bkgd_slice_spec0)[0], np.shape(bkgd_slice_spec0)[1])
#SIGNAL1
res_diff_spec1 = difference(res_slice_spec1, np.shape(res_slice_spec1)[0], np.shape(res_slice_spec1)[1])
bkgd_diff_spec1 = difference(bkgd_slice_spec1,np.shape(bkgd_slice_spec1)[0], np.shape(bkgd_slice_spec1)[1])

### Complex Differencing

In [ ]:
np.imag(spectrum0)

In [ ]:
#27-02-2020
#New differencing using complex parts 


'''(np.real(specgram[i,j+1])-np.real(specgram[i,j]))^2 + 
(np.imag(specgram[i,j+1])-np.imag(specgram[i,j]))^2''' #from Siong


for i in range(np.shape(res_slice_spec0)[0]):
    for j in range(np.shape(res_slice_spec0)[1]-1):
      
        diff_spec_complex[i,j] = (np.real(res_slice_spec0[i,j+1])-np.real(res_slice_spec0[i,j]))^2 + (np.imag(res_slice_spec0[i,j+1])-np.imag(res_slice_spec0[i,j]))^2



### Histogram of differenced and undifferenced data

In [ ]:
#slices histogram

def slice_hist(spectrogram, title):
    

    #want to plot all pixels so flatten array to 1D
    spectrogram = np.array(spectrogram).flatten()

    #Histogram of logN vs diff(f,t) - (N counts)
    n, bins, patches = plt.hist(x=spectrogram, bins='auto', log = True)
    
    plt.xlabel('d(f,t)')
    plt.ylabel('log(N)')
    plt.xlim(-1,40)
    
    plt.title(title,fontsize=23)
    return n, bins

In [ ]:
#SIGNAL 0
#plot hists of both slices, differenced and undifferenced
plt.figure(figsize=(25,12))

#1-resonant mode slice, undifferenced
plt.subplot(221)
slice_hist(res_slice_spec0, title = 'Undifferenced Spectrogram 0: -2 to -3Hz Resonant Mode');

#2-resonant mode slice, differenced
plt.subplot(222)
slice_hist(res_diff_spec0, title = 'Differenced Spectrogram 0: -2 to -3Hz Resonant Mode');

#3-background slice, undifferenced
plt.subplot(223)
slice_hist(bkgd_slice_spec0, title = 'Undifferenced Spectrogram 0: Background');

#4-background slice, undifferenced
plt.subplot(224)
slice_hist(bkgd_diff_spec0, title = 'Differenced Spectrogram 0: Background');

In [ ]:
#SIGNAL 1
#plot hists of both slices, differenced and undifferenced
plt.figure(figsize=(25,12))

#1-resonant mode slice, undifferenced
plt.subplot(221)
slice_hist(res_slice_spec1, title = 'Undifferenced Spectrogram 1: 2 to 3Hz Resonant Mode');

#2-resonant mode slice, differenced
plt.subplot(222)
slice_hist(res_diff_spec1, title = 'Differenced Spectrogram 1: 2 to 3Hz Resonant Mode');

#3-background slice, undifferenced
plt.subplot(223)
slice_hist(bkgd_slice_spec1, title = 'Undifferenced Spectrogram 1: Background');

#4-background slice, undifferenced
plt.subplot(224)
slice_hist(bkgd_diff_spec1, title = 'Differenced Spectrogram 1: Background');